# Preparation (run before optimization)

This file creates the S2F2D.csv file which contains all input data for the optimization. It only needs to be run once for the optimization.

## Table of Contents

1. Preparation
2. Generate a grid over Europe for all candidate positions of facilities and save in Facilities.xlsx
3. Create the Distance Matrix for Supply-to-Facility points and Facility-to-Demand points
* 3a) Supply to Facility (Supply = origin, Facility = destination)
* 3b) Facility to Demand (Facility = origin, Demand = destination)
* 3c) Create the S2F2D dataframe for the optimization
* 3d) Merge S2F2D with the Fixed cost
* 3e) Calculate Transport Costs and add to S2F2D

## 1. Preparation

### Import libraries (install if necessary)

In [1]:
# (uncomment install lines if necessary)
#import kaleido
#kaleido.__version__ #0.2.1
import numpy as np
import pandas as pd
import json
import geopandas as gpd
#import folium
import plotly.express as px
#import plotly.graph_objects as go
import requests
from global_land_mask import globe # python -m pip install global_land_mask
import time
import constantsfile # constantsfile.py

### Load in the Supply and Demand points from Excel files

In [2]:
df_demand = pd.read_excel("Demand.xlsx", usecols=["Name", 
                                                  "Country",
                                                  "Latitude",
                                                  "Longitude",
                                                  "Tonnes"])
df_supply = pd.read_excel("Supply.xlsx", usecols=["Name",
                                                  "Country",
                                                  "Latitude",
                                                  "Longitude",
                                                  "Tonnes"])

## 2. Generate a grid over Europe for all candidate positions of facilities and save in Facilities.xlsx

In [3]:
# Generate the grid coordinates
grid = []
for lat in range(int(constantsfile.min_lat), int(constantsfile.max_lat) + 1):
    for lon in range(int(constantsfile.min_lon), int(constantsfile.max_lon) + 1):
        if globe.is_land(lat, lon):  # Check if coordinates are on land and include if yes
            for i in range(int(constantsfile.grid_spacing)):
                for j in range(int(constantsfile.grid_spacing)):
                    point_lat = lat + i / constantsfile.grid_spacing
                    point_lon = lon + j / constantsfile.grid_spacing
                    params = { 
                        'key': constantsfile.api_key_opencage,
                        'q': f"{point_lat},{point_lon}",
                        'no_annotations': 1
                    } # API to check the country the coordinates are in
                    response = requests.get(constantsfile.base_url_opencage, params=params).json()
                    if 'results' in response and len(response['results']) > 0:
                        country = response['results'][0]['components']['country']
                        grid.append((point_lat, point_lon, country))

# Create a DataFrame with the grid coordinates and country information
df_facilities = pd.DataFrame(grid, columns=['Latitude', 
                                            'Longitude',
                                            'Country'])

# Reorder the columns
df_facilities = df_facilities[['Country',
                               'Latitude', 
                               'Longitude']].reset_index()

# Rename the 'index' column
df_facilities = df_facilities.rename(columns={'index': 'Name'})

# Replace name of countries so they match the Fixed Cost df
df_facilities['Country'] = df_facilities['Country'].replace('Czechia','Czech Republic')

# Add a prefix "N" to the Facility name number
df_facilities['Name'] = 'N' + df_facilities['Name'].astype(str)

# Save the DataFrame to an Excel file
df_facilities.to_excel('Facilities.xlsx', index=True)

In [4]:
df_facilities = pd.read_excel("Facilities.xlsx", usecols=["Name", "Country", "Latitude","Longitude"])

fig = px.scatter_mapbox(df_facilities,
                        lat ="Latitude",lon="Longitude",
                        hover_name="Name",
                        zoom=2.3, center=dict(lon=15, lat=56),
                        height=600, width=800,
                        opacity=0.8,
                        color_continuous_scale=px.colors.sequential.algae,
                        )
fig.update_layout(mapbox_style='light', mapbox_accesstoken=constantsfile.mapbox_access_token)
fig.update_layout(title_text="Facilities Grid")
fig.update_traces(marker=dict(size=10))
fig.show()
#fig.write_image("Facility.png", scale=1, engine='kaleido')

## 3. Create the Distance Matrix for Supply-to-Facility points and Facility-to-Demand points

### 3a) Supply to Facility (Supply = origin, Facility = destination)

Chunk the Supply dataframe into 30-row sub-dataframes as a work-around to not exceed the openrouteservice maximums limits. Each chunk will be fed into the script in a loop.

In [5]:
#df_origins_full = supply (Actually reverse route in this case. 2030 is chosen as it includes all nodes.)
df_origins_full = df_supply
df_destinations_full = df_facilities

# Function split_dataframe
def split_dataframe(df, chunk_size):
    num_chunks = len(df) // chunk_size
    if len(df) % chunk_size != 0:
        num_chunks += 1

    dfs = []
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size
        dfs.append(df[start_idx:end_idx].reset_index(drop=True))

    return dfs

# Chunk size 30
df_destinations_full = split_dataframe(df_destinations_full, 30)

Create a new pandas dataframe and combine name, country, and coordinate columns into a new column for identification to be possible after the API matrix run. Run the chunks throught the API in a loop and then combine them. Then create a file to save the result.

In [6]:
# Loop over each chunk
distances = []
for j in range(len(df_destinations_full)):
  df_destinations = df_destinations_full[j]
  df_origins = df_origins_full

  # Temporarily convert the latitude and longitude as strings to combine wth name and country
  df_destinations['Latitude'] = df_destinations['Latitude'].astype(str)
  df_destinations['Longitude'] = df_destinations['Longitude'].astype(str)
  df_origins['Latitude'] = df_origins['Latitude'].astype(str)
  df_origins['Longitude'] = df_origins['Longitude'].astype(str)
  df_origins['Tonnes'] = df_origins['Tonnes'].astype(str)

  # Combine name and country to feed in as the identifier of the coordinates for the API
  df_destinations['name&country&coordinates'] = df_destinations['Name'] + ';' + df_destinations['Country'] + ';' + df_destinations['Latitude'] + ';' + df_destinations['Longitude']
  df_origins['name&country&coordinates&tonnes'] = df_origins['Name'] + ';' + df_origins['Country'] + ';' + df_origins['Latitude'] + ';' + df_origins['Longitude'] + ';' + df_origins['Tonnes']

  # Convert coordinate columns back to float
  df_destinations['Latitude'] = df_destinations['Latitude'].astype(float)
  df_destinations['Longitude'] = df_destinations['Longitude'].astype(float)
  df_origins['Latitude'] = df_origins['Latitude'].astype(float)
  df_origins['Longitude'] = df_origins['Longitude'].astype(float)

  # Reorder and drop unecessary columns
  df_destinations = df_destinations[['name&country&coordinates', 
                                     'Latitude', 
                                     'Longitude']]
  df_origins = df_origins[['name&country&coordinates&tonnes',
                           'Latitude', 
                           'Longitude']]

  # Reset index
  df_origins = df_origins.reset_index()
  df_destinations = df_destinations.reset_index()

  # Combine the origins and destination dataframes into one
  df_locations = pd.concat([df_origins, df_destinations], ignore_index=True)

  # Convert the df coordinates into API format
  locations = []
  for _, row in df_locations.iterrows():
      locations.append([row['Longitude'], row['Latitude']])

  # The origins and destinations in the API are expressed in terms of a list of
  # the origins and destinations dataframes, so here they are converted to a list
  origins = []
  for _, row in df_origins.iterrows():
      origins.append([row['Longitude'], row['Latitude']])
  destinations = []
  for _, row in df_destinations.iterrows():
      destinations.append([row['Longitude'], row['Latitude']])

  # Create a dictionary to represent the matrix parameters
  body = {
      'locations': locations,
      'destinations': list(range(len(origins), len(origins) + len(destinations))),
      'metrics': ['distance'],
      'sources': list(range(len(origins))),
      'units': 'km'
  }

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': constantsfile.api_key_openrouteservice,
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post(constantsfile.base_url_openrouteservice, json=body, headers=headers)
  call_json = call.json()

  for k, row in enumerate(call_json['distances']):
    for l, distance in enumerate(row):
        distances.append({
            'origin': df_origins.loc[k, 'name&country&coordinates&tonnes'],
            'destination': df_destinations.loc[l, 'name&country&coordinates'],
            'distance_km': distance
          })

distance_df = pd.DataFrame(distances)

# Write the DataFrame to Excel
distance_df.to_excel('Supply_to_Facility_distances.xlsx', index=False)

In [7]:
print("Waiting for 60 seconds to make sure API doesn't get overloaded.")
time.sleep(60)
print("60 seconds have passed, moving on to next part of the script.")

Waiting for 60 seconds to make sure API doesn't get overloaded.
60 seconds have passed, moving on to next part of the script.


### 3b) Facility to Demand (Facility = origin, Demand = destination)

Chunk the Supply dataframe into 30-row sub-dataframes as a work-around to not exceed the openrouteservice maximums limits. Each chunk will be fed into the script in a loop.


In [8]:
df_origins_full = df_demand
df_destinations_full = df_facilities

# Function split_dataframe
def split_dataframe(df, chunk_size):
    num_chunks = len(df) // chunk_size
    if len(df) % chunk_size != 0:
        num_chunks += 1

    dfs = []
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size
        dfs.append(df[start_idx:end_idx].reset_index(drop=True))

    return dfs

# Chunk size 30
df_destinations_full = split_dataframe(df_destinations_full, 30)

Create a new pandas dataframe and combine name, country, and coordinate columns into a new column for identification to be possible after the API matrix run. Run the chunks throught the API in a loop and then combine them. Then create a file to save the result.

In [9]:
# Loop over each chunk
distances = []
for j in range(len(df_destinations_full)):
  df_destinations = df_destinations_full[j]
  df_origins = df_origins_full

  # Temporarily convert the latitude, longitude, and volume as strings to combine with name and country
  df_destinations['Latitude'] = df_destinations['Latitude'].astype(str)
  df_destinations['Longitude'] = df_destinations['Longitude'].astype(str)
  df_origins['Latitude'] = df_origins['Latitude'].astype(str)
  df_origins['Longitude'] = df_origins['Longitude'].astype(str)
  df_origins['Tonnes'] = df_origins['Tonnes'].astype(str)

  # Combine name and country to feed in as the identifier of the coordinates for the API
  df_destinations['name&country&coordinates'] = df_destinations['Name'] + ';' + df_destinations['Country'] + ';' + df_destinations['Latitude'] + ';' + df_destinations['Longitude']
  df_origins['name&country&coordinates&tonnes'] = df_origins['Name'] + ';' + df_origins['Country'] + ';' + df_origins['Latitude'] + ';' + df_origins['Longitude'] + ';' + df_origins['Tonnes']

  # Convert coordinate columns back to float for use in API
  df_destinations['Latitude'] = df_destinations['Latitude'].astype(float)
  df_destinations['Longitude'] = df_destinations['Longitude'].astype(float)
  df_origins['Latitude'] = df_origins['Latitude'].astype(float)
  df_origins['Longitude'] = df_origins['Longitude'].astype(float)

  # Reorder and drop unecessary columns
  df_destinations = df_destinations[['name&country&coordinates', 
                                     'Latitude', 
                                     'Longitude']]
  df_origins = df_origins[['name&country&coordinates&tonnes', 
                           'Latitude', 
                           'Longitude']]

  # Reset index
  df_origins = df_origins.reset_index()
  df_destinations = df_destinations.reset_index()

  # Combine the origins and destination dataframes into one
  df_locations = pd.concat([df_origins, df_destinations], ignore_index=True)

  # Convert the df coordinates into API format
  locations = []
  for _, row in df_locations.iterrows():
      locations.append([row['Longitude'], row['Latitude']])

  # The origins and destinations in the API are expressed in terms of a list of
  # the origins and destinations dataframes, so here they are converted to a list
  origins = []
  for _, row in df_origins.iterrows():
      origins.append([row['Longitude'], row['Latitude']])
  destinations = []
  for _, row in df_destinations.iterrows():
      destinations.append([row['Longitude'], row['Latitude']])

  # Create a dictionary to represent the matrix parameters
  body = {
      'locations': locations,
      'destinations': list(range(len(origins), len(origins) + len(destinations))),
      'metrics': ['distance'],
      'sources': list(range(len(origins))),
      'units': 'km'
  }

  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': constantsfile.api_key_openrouteservice,
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post(constantsfile.base_url_openrouteservice, json=body, headers=headers)
  call_json = call.json()

  for k, row in enumerate(call_json['distances']):
    for l, distance in enumerate(row):
        distances.append({
            'origin': df_origins.loc[k, 'name&country&coordinates&tonnes'],
            'destination': df_destinations.loc[l, 'name&country&coordinates'],
            'distance_km': distance
          })


distance_df = pd.DataFrame(distances)

# Write the DataFrame to Excel
distance_df.to_excel('Facility_to_Demand_distances.xlsx', index=False)

### 3c) Create the S2F2D dataframe for the optimization from Supply to Facility (S2F) and Facility to Demand Nodes (F2D)

In [10]:
### Calculate Distances from Supply to Facility Nodes (S2F) ###

# Read in Excel files as dataframes
df_S2F = pd.read_excel("Supply_to_Facility_distances.xlsx", sheet_name="Sheet1")
df_S2F_coordinates = pd.DataFrame().assign(
    origin = df_S2F["origin"],
    destination = df_S2F["destination"],
    distance_km = df_S2F["distance_km"])

# Split the origin and destination columns into separate columns
df_S2F_coordinates[['origin_name', 
                    'origin_country',
                    'origin_latitude',
                    'origin_longitude',
                    'origin_tonnes']] = df_S2F_coordinates['origin'].str.split(';', expand=True)
df_S2F_coordinates[['destination_name', 
                    'destination_country',
                    'destination_latitude',
                    'destination_longitude']] = df_S2F_coordinates['destination'].str.split(';', expand=True)

# Reorder the columns and rename column headers to make them different from F2D
df_S2F_coordinates = df_S2F_coordinates[['origin_name', 
                                         'origin_country',
                                         'origin_latitude',
                                         'origin_longitude',
                                         'origin_tonnes',
                                         'destination_name', 
                                         'destination_country',
                                         'destination_latitude',
                                         'destination_longitude',
                                         'distance_km']].rename(columns={'origin_name': 'Supply_name',
                                                                'origin_country':'Supply_country',
                                                                'origin_latitude':'Supply_latitude',
                                                                'origin_longitude':'Supply_longitude',
                                                                'origin_tonnes':'Supply_tonnes',
                                                                'destination_name':'Facility_name',
                                                                'destination_country':'Facility_country',
                                                                'destination_latitude':'Facility_latitude',
                                                                'destination_longitude':'Facility_longitude',
                                                                'distance_km':'S2F_distance_km'})

# Remove empty rows that don't have distances
df_S2F_coordinates = df_S2F_coordinates.dropna(subset = ['S2F_distance_km'])

### Calculate Distances from Facility to Demand Nodes (F2D) ###

# Read in Excel files as dataframes
df_F2D = pd.read_excel("Facility_to_Demand_distances.xlsx", sheet_name="Sheet1")
df_F2D_coordinates = pd.DataFrame().assign(
    origin = df_F2D["origin"],
    destination = df_F2D["destination"],
    distance_km = df_F2D["distance_km"])

# Split the origin and destination columns into separate columns
df_F2D_coordinates[['origin_name', 
                    'origin_country',
                    'origin_latitude',
                    'origin_longitude', 
                    'origin_tonnes']] = df_F2D_coordinates['origin'].str.split(';', expand=True)
df_F2D_coordinates[['destination_name', 
                    'destination_country',
                    'destination_latitude',
                    'destination_longitude']] = df_F2D_coordinates['destination'].str.split(';', expand=True)

# Reorder the columns and rename column headers to make them different from S2F
df_F2D_coordinates = df_F2D_coordinates[['origin_name', 
                                         'origin_country',
                                         'origin_latitude',
                                         'origin_longitude',
                                         'origin_tonnes',
                                         'destination_name', 
                                         'destination_country',
                                         'destination_latitude',
                                         'destination_longitude',
                                         'distance_km']].rename(columns={'origin_name': 'Demand_name',
                                                                'origin_country':'Demand_country',
                                                                'origin_latitude':'Demand_latitude',
                                                                'origin_longitude':'Demand_longitude',
                                                                'origin_tonnes':'Demand_tonnes',
                                                                'destination_name':'Facility_name',
                                                                'destination_country':'Facility_country',
                                                                'destination_latitude':'Facility_latitude',
                                                                'destination_longitude':'Facility_longitude',
                                                                'distance_km':'F2D_distance_km'})

# Remove empty rows that don't have distances
df_F2D_coordinates = df_F2D_coordinates.dropna(subset = ['F2D_distance_km'])

# Remove columns that are the same as S2F
df_F2D_coordinates = df_F2D_coordinates.drop(columns=['Facility_country',
                                                      'Facility_latitude',
                                                      'Facility_longitude'])

### MERGE F2D, S2F AND FIXES COSTS ###

# Merge D2F and S2F
S2F2D = pd.merge(df_F2D_coordinates, df_S2F_coordinates,  how='left', left_on=['Facility_name'], right_on = ['Facility_name']).reset_index()

### 3d) Merge S2F2D with the Fixed cost

In [11]:
df_fixed_costs = pd.read_excel("Fixed_costs.xlsx", usecols=["Country", "Fixed_cost"])

# Do an inner join between df_facilities and df_fixed_costs and drop the Latitude and Longitude columns
df_facilityCoordinatesFull = pd.merge(df_facilities, 
                                      df_fixed_costs, 
                                      on='Country', 
                                      how='inner').drop(columns=['Latitude',
                                                                 'Longitude']).reset_index()

# Remove rows with no country (only if countries where candidate facilities don't exist shouldn't be included)
df_facilityCoordinatesFull = df_facilityCoordinatesFull[df_facilityCoordinatesFull.Country != ""]

# Merge S2F2D with fixed costs and remove generated index columns and repeated facility name column.
S2F2D_FC = pd.merge(S2F2D, 
                    df_facilityCoordinatesFull,  
                    how='inner', 
                    left_on=['Facility_name'], 
                    right_on = ['Name']).drop(columns=['index_x',
                                                       'index_y',
                                                       'Name']).reset_index(drop = True)

### 3e) Calculate Transport Costs and add to S2F2D

In [12]:
# Create new columns for tranportation cost (cost per tonne)
S2F2D_FC['Transport_cost_F2D'] = S2F2D_FC['F2D_distance_km']*constantsfile.drivingcost_per_tonnekm
S2F2D_FC['Transport_cost_S2F'] = S2F2D_FC['S2F_distance_km']*constantsfile.drivingcost_per_tonnekm

# Save S2F2D_FC as CSV file
S2F2D_FC.to_csv('S2F2D.csv', index=False)